In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [59]:
len_data_arr = []
results_data_arr_agglo = []
results_data_arr_bunch = []
for root, dirs, files in os.walk('Results_21102020'):
    for file in files:
        #print(file)
        #print(file.split('_')[-1])
        if file.split('_')[-1] == 'len.txt':
            #print(file, 'len')
            proj_len_file = open('Results_21102020/' + file)
            proj_name = file.split('_')[0]
            proj_len = proj_len_file.readline().replace('\n','')
            len_data_arr.append([proj_name, proj_len])
            proj_len_file.close()
        elif file.split('_')[-1] == 'results.txt':
            #print(file, 'results')
            proj_result_file = open('Results_21102020/' + file)
            proj_name = file.split('_')[0]
            
            initial_line = 2 - 1
            initial_line_result = 5 - 1
            tmp_arr = []
            
            for i, line in enumerate(proj_result_file):
                
                if i == initial_line:
                    line = line.replace('MoJo', '')
                    line = line.replace('.rsf', '')
                    line = line.replace('(', '')
                    line = line.replace(')', '')
                    line = line.split('_')
                    name = line[1].split('/')[-1]
                    #version = line[0].split('-')[-1]
                    version = line[2]
                    #print(line)
                    ## If gathers results for bunch
                    if line[3] == 'exhaustive' or line[3] == 'hillclimbing' or line[3] == 'ga':
                        bunch_bool = True
                        method = line[3]
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(method)
                        
                        initial_line += 5
                        
                    
                    
                    ## Else gathers results for agglomerative
                    else:
                        agglo_bool = True
                        n_cluster = line[3]
                        affinity = line[4]
                        linkage = line[5]
                        #print(line)
                        #print('Version', version)
                        initial_line += 5
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(n_cluster)
                        tmp_arr.append(affinity)
                        tmp_arr.append(linkage)
                        
                elif i == initial_line_result:
                    n_MoJo = line.split(' ')[-1].split('\n')[0]

                    #print(line)
                    initial_line_result += 5
                    tmp_arr.append(n_MoJo)
                    #tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
                    
                    if agglo_bool == True:
                        results_data_arr_agglo.append(tmp_arr)
                        agglo_bool = False
                    elif bunch_bool == True:
                        results_data_arr_bunch.append(tmp_arr)
                        bunch_bool = False
                    tmp_arr = []
                    
                    
                
                

In [60]:
pd.DataFrame(results_data_arr_agglo)

,0,1,2,3,4,5
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695
...,...,...,...,...,...,...
62728,web3j-web3j,v4.5.2,25,manhattan,average,66
62729,web3j-web3j,v4.5.2,25,manhattan,single,59
62730,web3j-web3j,v4.5.2,25,cosine,complete,205
62731,web3j-web3j,v4.5.2,25,cosine,average,251


In [61]:
pd.DataFrame(results_data_arr_bunch)

,0,1,2,3
0,aws-aws-sdk-java-v2,2.9.9,hillclimbing,1421
1,aws-aws-sdk-java-v2,2.9.9,ga,1646
2,aws-aws-sdk-java-v2,2.9.9,exhaustive,1674
3,aws-aws-sdk-java-v2,2.9.8,hillclimbing,1406
4,aws-aws-sdk-java-v2,2.9.8,ga,1722
...,...,...,...,...
2336,web3j-web3j,v4.5.4,ga,378
2337,web3j-web3j,v4.5.4,exhaustive,373
2338,web3j-web3j,v4.5.2,hillclimbing,350
2339,web3j-web3j,v4.5.2,ga,362


In [62]:
agglo_results = pd.DataFrame(results_data_arr_agglo)
agglo_results.columns = ['project_name', 'version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results']
agglo_results.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695


In [ ]:
bunch_results = pd.DataFrame(results_data_arr_agglo)
bunch_results.columns = ['project_name', 'version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results']
bunch_results.head()